In [1]:
import cv2
import numpy as np
import time

#Load YOLO
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg") 
#net = cv2.dnn.readNet("yolov3-tiny.weights","yolov3-tiny.cfg") #Tiny Yolo
classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

In [2]:
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [3]:
img =  cv2.imread("dog.jpg")


In [4]:
blob =cv2.dnn.blobFromImage(img,1/255, (320,320),(0,0,0),swapRB=True ,crop=False)

In [5]:
net.setInput(blob)

In [6]:
output_layer_name = net.getUnconnectedOutLayersNames()
layeroutput = net.forward(output_layer_name)

In [7]:
colors= np.random.uniform(0,255,size=(len(classes),3))

In [8]:
import cv2
import numpy as np
import face_recognition
import os
import pyttsx3
from datetime import datetime
from sqlite3 import connect
import re
engine = pyttsx3.init()


In [9]:

while(True):
    print("""ENTER YOUR OPTION --- 
          1.object Detection
          
          2.Face Recognition
          3.New Enrollement
          4.Exit
          """)
    abc = int(input("enter your choice"))
    if abc==1:
        cap=cv2.VideoCapture(0)
        
        # url="https://192.168.1.101:8080/video"
        # cap.open(url)#0 for 1st webcam
        font = cv2.FONT_HERSHEY_PLAIN
        starting_time= time.time()
        frame_id = 0

        while True:
            _,frame= cap.read() # 
            frame_id+=1

            height,width,channels = frame.shape
            #detecting objects
            blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False) #reduce 416 to 320    


            net.setInput(blob)
            outs = net.forward(output_layer_name)
            #print(outs[1])


            class_ids=[]
            confidences=[]
            boxes=[]
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.3:
                        #onject detected
                        center_x= int(detection[0]*width)
                        center_y= int(detection[1]*height)
                        w = int(detection[2]*width)
                        h = int(detection[3]*height)

                        #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
                        #rectangle co-ordinaters
                        x=int(center_x - w/2)
                        y=int(center_y - h/2)
                        #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

                        boxes.append([x,y,w,h]) 
                        confidences.append(float(confidence)) 
                        class_ids.append(class_id) 

            indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)


            for i in range(len(boxes)):
                if i in indexes:
                    x,y,w,h = boxes[i]
                    label = str(classes[class_ids[i]])
                    confidence= confidences[i]
                    color = colors[class_ids[i]]
                    cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
                    cv2.putText(frame,label+" "+str(round(confidence,2))+" "+str(x)+" X "+str(y+30),(x,y+30),font,1,(255,255,255),2)
                    print(label+" : "+str(round(confidence,2))+" "+str(x)+" X "+str(y+30),(x,y+30))
                    confidence = str(round(confidence,2)*100)
                    engine.say(label + "with" + confidence + "% confidentiality")
                    engine.runAndWait()

            elapsed_time = time.time() - starting_time
            fps=frame_id/elapsed_time
            cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),font,2,(0,0,0),1)

            cv2.imshow("Image",frame)
            key = cv2.waitKey(1) 

            if cv2.waitKey(10)==13:
                break

        cap.release()    
        cv2.destroyAllWindows()   

    if(abc==2):
        #reading Images
        
        path='E:/notes/SY/2nd_sem/SDP/images'
        images=[]
        personNAme=[]
        myList=os.listdir(path)
        for cur_img in myList:
            personNAme.append(cur_img.split(".")[0])
            images.append(cv2.imread(f'{path}/{cur_img}'))
        print(personNAme)

        #performs encoding on face
        def faceEncodings(images):
            encodeList=[]
            for img in images:
                img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                encode=face_recognition.face_encodings(img)[0]
                encodeList.append(encode)
            return encodeList
        encodeListKnow=faceEncodings(images)
        print("Encodings Completed!!!")

        #mark attendance
        def attendance(name):
            with open('E:/notes/SY/2nd_sem/SDP/attendance.csv','r+') as f:
                myDataList=f.readlines()
                nameList=[]
                for line in myDataList:
                    entry=line.split(',')
                    nameList.append(entry[0])

                if name not in nameList:
                    time_now=datetime.now()
                    tStr=time_now.strftime('%H:%M:%S')
                    dStr=time_now.strftime('%d/%m/%Y')
                    f.writelines(f'\n{name},{tStr},{dStr}')
                    print("Student name : "+name)


        #image processing with camera
        cap=cv2.VideoCapture(0)

#         url="https://192.168.1.101:8080/video"
#         cap.open(url)
        while True:
            ret,frame=cap.read()
            faces=cv2.resize(frame,(0,0),None,0.25,0.25)
            faces=cv2.cvtColor(faces,cv2.COLOR_BGR2RGB)

            facesCurrentFrame=face_recognition.face_locations(faces)
            encodesCurrentFrame=face_recognition.face_encodings(faces,facesCurrentFrame)

            for encodeFace,faceLoc in zip(encodesCurrentFrame,facesCurrentFrame):
                matches=face_recognition.compare_faces(encodeListKnow,encodeFace)
                faceDis=face_recognition.face_distance(encodeListKnow,encodeFace)

                matchIndex=np.argmin(faceDis)

                if matches[matchIndex]:
                    name=personNAme[matchIndex].upper()
                    # print(name)
                    y1,x2,y2,x1=faceLoc
                    y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
                    cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
                    cv2.rectangle(frame,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
                    cv2.putText(frame,name+" "+str(x2-x1)+" X "+str(y2-y1),(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,275),2)
                    attendance(name)


            cv2.imshow("Camera",frame)
            if cv2.waitKey(10)==13:
                break

        cap.release()
        cv2.destroyAllWindows()

#         cv2.waitKey(1)
    if abc==3:
        num=int(input("\n1.Enter the record for the students\n2.Capture Student Image"))
        if num==1:
            
            con = connect("Database_student.db")
            x=con.cursor()
#             x.execute("CREATE TABLE STUDENTRECORD(ID NUMBER(3) PRIMARY KEY ,NAME VARCHAR(20),LAST_NAME VARCHAR(20),CLASS NUMBER(3),ADDRESS VARCHAR(50),EMAIL_ID VARCHAR(20));")
            regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
            con = connect("Database_student.db")
            x=con.cursor()
            regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
            f=int(input("HOW MANY RECORD YOU WANT TO INSERT : "))
            for i in range(f):
                s_id=int(input("id :"))
                s_name=input("name :")
                s_last=input("Last name : ")
                s_class=input("Class :")
                s_Add=input("Address of Student : ")
                s_Email=input("Email Id :")
                if(re.match(regex, s_Email)):
                    print("Valid Email")
        if num==2:
            cam = cv2.VideoCapture(0)
            #cv2.namedWindow("Webcam")
#             url="https://192.168.1.101:8080/video"
#             cam.open(url)
            while True:
                ret,frame=cam.read()
                if not ret :
                    print("Camera Closed ")
                    break
                cv2.imshow("Web",frame)
                print("If you want to capture images click space button ")
                print("else click Esc bu image ")
                n=input("Enter the name :tton")
                print("Enter your name to save ")
                print("********** Please press Space buton to save the picture **********")
                k=cv2.waitKey(0)
                if k%256==27:
                    print("ESC it ")
                    break
                elif k%256==32:
                    directory = r'E:/notes/SY/2nd_sem/SDP/images'
                    os.chdir(directory)
                    n = "{}.jpg".format(n)
                    save = cv2.imwrite(n,frame)
                    print("image saved")

                cam.release()
                cv2.destroyAllWindows()
            else:
                print("Exit")
                break
    if abc==4:
        print("THANK YOU!!!!")
        break
    else:
        print("type correct option")

ENTER YOUR OPTION --- 
          1.object Detection
          
          2.Face Recognition
          3.New Enrollement
          4.Exit
          
['Dhamma', 'Johnny', 'mustafa', 'mustan', 'SHree', 'tanishq', 'Tej']
Encodings Completed!!!
type correct option
ENTER YOUR OPTION --- 
          1.object Detection
          
          2.Face Recognition
          3.New Enrollement
          4.Exit
          
type correct option
ENTER YOUR OPTION --- 
          1.object Detection
          
          2.Face Recognition
          3.New Enrollement
          4.Exit
          
THANK YOU!!!!
